In [56]:
import numpy as np
import pandas as pd
import nltk
from sklearn.preprocessing import OneHotEncoder as oneHot
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.initializers import he_uniform, glorot_uniform
from tensorflow.keras.metrics import AUC
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
#Read the train and test datasets with column names as target and text
train= pd.read_csv('atis_intents_train.csv',names= ["target", "text"])
test= pd.read_csv('atis_intents_test.csv',names= ["target", "text"])

In [58]:
train.shape

(4834, 2)

In [59]:
test.shape

(800, 2)

In [60]:
train.head(10) #Get Top 10 rows from train dataset

,target,text
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...
5,atis_flight,i need a flight tomorrow from columbus to min...
6,atis_aircraft,what kind of aircraft is used on a flight fro...
7,atis_flight,show me the flights from pittsburgh to los an...
8,atis_flight,all flights from boston to washington
9,atis_ground_service,what kind of ground transportation is availab...


In [61]:
train['target'].value_counts()

target
atis_flight            3666
atis_airfare            423
atis_ground_service     255
atis_airline            157
atis_abbreviation       147
atis_aircraft            81
atis_flight_time         54
atis_quantity            51
Name: count, dtype: int64

In [62]:
encode_target= oneHot().fit(np.array(train.target).reshape(-1,1))

In [63]:
train_target_encoded= encode_target.transform(np.array(train.target).reshape(-1,1)).toarray()
test_target_encoded= encode_target.transform(np.array(test.target).reshape(-1,1)).toarray()

In [64]:
train_target_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [65]:
test_target_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [66]:
train["text"]= train.text.map(lambda l: l.lower())
test["text"]= test.text.map(lambda l: l.lower())

In [67]:
train.head()

,target,text
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [68]:
test.head()

,target,text
0,atis_flight,i would like to find a flight from charlotte ...
1,atis_airfare,on april first i need a ticket from tacoma to...
2,atis_flight,on april first i need a flight going from pho...
3,atis_flight,i would like a flight traveling one way from ...
4,atis_flight,i would like a flight from orlando to salt la...


In [69]:
train["text"]= train.text.map(word_tokenize)
test["text"]= test.text.map(word_tokenize)

In [70]:
train["text"]

0       [i, want, to, fly, from, boston, at, 838, am, ...
1       [what, flights, are, available, from, pittsbur...
2       [what, is, the, arrival, time, in, san, franci...
3          [cheapest, airfare, from, tacoma, to, orlando]
4       [round, trip, fares, from, pittsburgh, to, phi...
                              ...                        
4829    [what, is, the, airfare, for, flights, from, d...
4830    [do, you, have, any, flights, from, denver, to...
4831    [which, airlines, fly, into, and, out, of, den...
4832    [does, continental, fly, from, boston, to, san...
4833    [is, there, a, delta, flight, from, denver, to...
Name: text, Length: 4834, dtype: object

In [71]:
def clean_data_rm_stop(strings, stop_list):
    sw= [str for str in strings if str not in stop_list]
    return sw
stop_words= stopwords.words("english")
rm_punc_stop= list(set(punctuation))+ stop_words #Remove punctuation and stop words

In [72]:
train["text"]= train.text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))
test["text"]= test.text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))

In [73]:
def normalize(text):
    return " ".join(text)
#We use PorterStemmer function from nltk.stem library.
stem_func= PorterStemmer()

train["text"]= train.text.map(lambda s: [stem_func.stem(x) for x in s])
train["text"]= train.text.apply(normalize)

test["text"]= test.text.map(lambda s: [stem_func.stem(x) for x in s])
test["text"]= test.text.apply(normalize)

In [74]:
# We use Tokenizer from tensorflow.keras.preprocessing.text library
num_words=10000
text_tokenizer= Tokenizer(num_words)
text_tokenizer.fit_on_texts(train.text) #fit_on_texts - creates the vocabulary index based on word frequency.

tokenized_train_data= text_tokenizer.texts_to_sequences(train.text) #Converting texts to sequences
tokenized_test_data= text_tokenizer.texts_to_sequences(test.text)

In [75]:
tokenized_train_data

[[42, 19, 2, 296, 17, 5, 376, 21],
 [1, 37, 8, 10, 45, 21],
 [17, 102, 4, 6, 377, 1, 12, 18],
 [32, 212, 142, 91],
 [36, 30, 16, 8, 11, 180, 134],
 [33, 1, 98, 174, 138],
 [164, 99, 108, 1, 100, 9],
 [3, 1, 8, 103, 104, 45],
 [1, 2, 18],
 [164, 27, 29, 37, 5],
 [3, 1, 9, 4, 6],
 [3, 1, 4, 78, 67, 43, 87],
 [32, 1, 2, 225],
 [1, 10, 113, 22],
 [3, 24, 31, 16, 2, 5],
 [3, 27, 29, 5],
 [1, 5, 8, 12, 113, 22, 132, 22],
 [33, 55, 1, 52, 12, 10, 9, 9, 2, 2, 10],
 [20, 41, 1, 2, 8, 45, 79, 204],
 [23, 14, 19, 5, 8, 53, 13],
 [3, 1, 4, 78, 67],
 [20, 15, 24, 31, 1, 53, 5, 10],
 [164, 190, 108, 39, 13],
 [28, 14, 55, 109, 5, 8, 7],
 [28, 14, 136, 1, 7, 5],
 [13, 89, 5, 8, 7],
 [3, 1, 2, 8, 25, 79, 204, 12, 2, 181, 69, 22],
 [7, 27, 29],
 [205, 33, 147, 9, 2, 17, 66],
 [3, 32, 36, 30, 16, 10, 9],
 [3, 1, 44, 46, 57, 171, 13, 84, 378],
 [1, 44, 46, 62],
 [20, 58, 1, 65, 11, 4, 6, 41, 1, 240],
 [33, 1, 9, 4, 6],
 [16, 1, 8, 11],
 [3, 13, 24, 31, 1],
 [1, 56, 139, 185, 60, 63],
 [28, 14, 187, 1, 8,

In [76]:
#We use pad_sequences from tensorflow.keras.preprocessing.sequence library
train_data= pad_sequences(tokenized_train_data, maxlen= 20, padding= "pre")
test_data= pad_sequences(tokenized_test_data, maxlen= 20, padding= "pre")

In [77]:
train_data

array([[  0,   0,   0, ...,   5, 376,  21],
       [  0,   0,   0, ...,  10,  45,  21],
       [  0,   0,   0, ...,   1,  12,  18],
       ...,
       [  0,   0,   0, ...,  13,  19,   5],
       [  0,   0,   0, ...,   6,  51,   5],
       [  0,   0,   0, ...,   5,   4,   6]], dtype=int32)

In [78]:
train_data.shape

(4834, 20)

In [79]:
def transform_matrix(data, tokenizer):
    output_shape_mat= [data.shape[0],
                  data.shape[1],
                  tokenizer.word_index.keys().__len__()] #Three dimensional matrix with samples, steps and number of uniques words as each dimension.
    results_data= np.zeros(output_shape_mat) #creates new array with given dimensions.

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            results_data[i, j, data[i,j]-1]= 1 # In this for loop, we are looping over the shape of the training & test data and assigning the cell of above created zero matrix to 1. We are performing encoding on the unique words to obtain the transformation matrix
    return results_data

trans_matrix_train= transform_matrix(train_data, text_tokenizer) #This will be the matrix on which the lstm model is applied
trans_matrix_test= transform_matrix(test_data, text_tokenizer) #This will be the matrix on which the model is tested

In [80]:
trans_matrix_train.shape #here 653 is the unique words corpus of our sentences means there are 653 unique words used in this dataset

(4834, 20, 654)

In [81]:
trans_matrix_test.shape

(800, 20, 654)

In [82]:
class lstm_model_class(object):
    def build_lstm_model(self,input_dimensions, op_shape, num_steps, dropout_rate, kernel_reg, bias_reg):
      ip_layer= Input(shape= (num_steps, input_dimensions)) #Define embedded layer with shape as number of steps and input dimensions. Note that both these are input variables to the model.

      lstm_model= LSTM(units= num_steps)(ip_layer) #Make the LSTM layer with number of steps as memory units
      dense_layer_1= Dense(op_shape, kernel_initializer= he_uniform(), #he_uniform draws samples in uniform distribution with -inf to +inf as range.
                   bias_initializer= "zeros",
                   kernel_regularizer= l2(l= kernel_reg),
                   bias_regularizer= l2(l= bias_reg))(lstm_model) # Create the  Dense layer which is the regular deeply connected layer
      int_layer= BatchNormalization()(dense_layer_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      int_layer= relu(int_layer) #This applies the rectified linear unit activation function
      int_layer= Dropout(rate= dropout_rate)(int_layer) #Dropout is used to define Dropout layer that sets input units to 0 with a frequency. Here it is dropout_rate
      output_1= Dense(op_shape, kernel_initializer= glorot_uniform(), #glorot_uniform draws samples in uniform distribution with stddev = sqrt(2 / (fan_in + fan_out)) fan_in is num of units in weight tensor and fan_out is num of output units
             bias_initializer= "zeros",
             kernel_regularizer= l2(l= kernel_reg),
             bias_regularizer= l2(l= bias_reg))(dense_layer_1) # Create another dense layer which is the output of the model.
      output_1= BatchNormalization()(output_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      final_output= softmax(output_1, axis= 1)

      loss_func= cce() # Since it is a multi-class classification problem, categorical crossentropy(cce) is used as the loss function
      perf_metrics= AUC() #our performance metric will be area under the curve
      optimizer= Adam() #we shall use Adam optimizer as our optimizer
      self.final_model= Model(inputs= [ip_layer], outputs= [final_output]) #Build the model with input and output layers
      self.final_model.compile(optimizer= optimizer, loss= loss_func, metrics= [perf_metrics]) #Compiling the keras model

    def train_lstm_model(self,x, y, valid_split, ep):
      self.final_model.fit(x, y, validation_split= valid_split, epochs= ep) #Create the train model


    def predict_lstm_model(self,x):
      return self.final_model.predict(x)    #Create the predict model


In [83]:
steps= trans_matrix_train.shape[1] #Define the number of steps is usually the number of steps in the train data.
input_dim= trans_matrix_train.shape[2] #Input dimension. Number of unique words in the train data
output_shape= train_target_encoded.shape[1] #Output shape. Usually the same number as the number of classes in the target variable. Here we have 8.
final_model= lstm_model_class()
final_model.build_lstm_model(input_dimensions= input_dim,
                  op_shape= output_shape,
                  num_steps= steps,
                  dropout_rate= 0.5, # Meaning 1 in 2 inputs will be randomly executed.
                  bias_reg= 0.3, # Reduce the bias in the model
                  kernel_reg= 0.3) #Reduce the weights excluding bias.



In [84]:
final_model.train_lstm_model(trans_matrix_train, train_target_encoded,
           0.2, 60) #Model takes train data, train target variable, validation split(here it is 80:20) and number of epochs.

Epoch 1/60
121/121 [==============================] - 6s 25ms/step - loss: 7.1767 - auc_1: 0.7524 - val_loss: 5.7953 - val_auc_1: 0.7679
Epoch 2/60
121/121 [==============================] - 3s 26ms/step - loss: 4.0845 - auc_1: 0.9337 - val_loss: 3.6591 - val_auc_1: 0.9572
Epoch 3/60
121/121 [==============================] - 2s 18ms/step - loss: 2.4576 - auc_1: 0.9742 - val_loss: 2.3497 - val_auc_1: 0.9582
Epoch 4/60
121/121 [==============================] - 2s 18ms/step - loss: 1.6018 - auc_1: 0.9881 - val_loss: 1.6067 - val_auc_1: 0.9407
Epoch 5/60
121/121 [==============================] - 3s 26ms/step - loss: 1.1616 - auc_1: 0.9918 - val_loss: 0.9094 - val_auc_1: 0.9941
Epoch 6/60
121/121 [==============================] - 2s 18ms/step - loss: 0.9167 - auc_1: 0.9940 - val_loss: 0.8409 - val_auc_1: 0.9932
Epoch 7/60
121/121 [==============================] - 3s 26ms/step - loss: 0.7937 - auc_1: 0.9947 - val_loss: 0.8501 - val_auc_1: 0.9849
Epoch 8/60
121/121 [=====================

In [85]:
pred_train= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_train)) #Predict on the train matrix and look at the performance
print(classification_report(train.target, pred_train)) #Print the classification report

152/152 [==============================] - 1s 7ms/step
                     precision    recall  f1-score   support

  atis_abbreviation       1.00      1.00      1.00       147
      atis_aircraft       0.94      1.00      0.97        81
       atis_airfare       1.00      0.99      0.99       423
       atis_airline       0.99      0.98      0.98       157
        atis_flight       1.00      1.00      1.00      3666
   atis_flight_time       1.00      0.07      0.14        54
atis_ground_service       1.00      1.00      1.00       255
      atis_quantity       0.53      1.00      0.69        51

           accuracy                           0.99      4834
          macro avg       0.93      0.88      0.85      4834
       weighted avg       0.99      0.99      0.98      4834



In [86]:
pred_test= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_test)) #Predict on the test data
print(classification_report(test.target, pred_test)) #Print the classification report

25/25 [==============================] - 0s 7ms/step
                     precision    recall  f1-score   support

  atis_abbreviation       1.00      1.00      1.00        33
      atis_aircraft       0.90      1.00      0.95         9
       atis_airfare       1.00      1.00      1.00        48
       atis_airline       0.88      0.92      0.90        38
        atis_flight       0.99      0.98      0.99       632
   atis_flight_time       0.00      0.00      0.00         1
atis_ground_service       1.00      0.94      0.97        36
      atis_quantity       0.38      1.00      0.55         3

           accuracy                           0.98       800
          macro avg       0.77      0.86      0.79       800
       weighted avg       0.98      0.98      0.98       800



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
